# Chapter 3: Data Engineering Fundamentals

## Data Sources

- User input: Stuff that users upload (text, images, videos, egc)

- System generated: logs, predictions etc
    - How to process logs
    - How to store logs

- Internal databases
    - How to query the databases on the fly? e.g. On Amazon, does query about "frozen" relate to the disney show or frozen food?

- Data
    - First party data: Data re: your users/customers
    - Second party data: data collected by another company made available to you
    - Third party data: public users who are not direct customers (e.g. Android's AAID, China's state supported CAID)


## Data formats

| Format | Binary/Text | Human-readable | Example use case |
| - | - | - | - |
| JSON | Text | Yes | Everywhere |
| CSV | Text | Yes | Everywhere |
| Parquet | Binary | No | Hadoop, Amazon Redshift |
| Avro | Binary primary | No | Hadoop |
| Protobuf | Binary primary | No | Google/Tensorflow |
| Pickle | Binary | No | Python/Pytorch Serialization |

- Row vs Column Major?
    - Just refers to whether consecutive objects are items in the same row, or column
    - Depending on this, you can have faster row access, or column access
    - For read heavy databases, you want faster column access (so you can only focus on data that you want)
    - For write heavy databases, you want faster row access (because when you write, each item is probably belonging to the same data row)

- Text vs Binary
    - Text is more readable, but less space efficient

## Data Models

### Relational Model

- Usually stored as CSV or parquet
- First normal form (1NF)
    - Every value is a single-valued attribute
    - Attribute domain doesn't change
    - Unique name for every attribute
    - Order does not matter
- Second normal form (2NF)
    - Reduce redundant data from getting stored (i.e. separate into separate table if there is duplicate data)
    - Downside is that more joins are needed, but upside is that less space is used

### NoSQL

- Relational DB is useful, but limited, because of the inflexibility of the schema
- Often when things kep changing, NoSQL works better
- This is most useful when your data comes in as self-contained documents, and relationships between documents are rare

- Document Model
    - Single continuous string (JSON, XML etc)
    - All documents have a unique key used for retrieval
    - All documents should have similar schema, but schema similarity is not enforced, giving you more flexibility
    - Easier to use, but harder to do stuff like `joins` etc


- Graph model
    - Data stored as nodes, with relationships modelled as edges
    - A picture is worth a thousand words so ![graphql](./artifacts/3_image.png) 

### Structured vs Unstructured

| Structured | Unstructured |
| - | - |
| Schema rigorously defined | No schema | 
| Easy to search/analyse | Fast Arrival | 
| Only handle data that follows schema | Handles data from any source | 
| Schema changes cause cascading failures | Offloads the problems with schema changes to downstream applications | 
| Stored in data warehouse | Stored in data lakes | 

## Data Storage Engines and Processing

### Transactional vs Analytical Processing

- If your app provides a service (ride matching, food ordering etc), you may wish to process and upload data each time a transaction is made. This is known as Online Transaction Processing (OLTP)
    - This usually requires low latency + high availability of tables
    - Most people expect databases to be ACID (Atomicity, Consistency, Isolation, Durability)
    - However, OLTP tables may wish to loosen ACID demands to meet the low latency + high availability requirements
    - Non ACID tables are sometimes known as BASE (Basically Available, Soft State, Eventual Consistency)

- OLTP tables, however, may not serve the need for analytics. For example, if I want to count the number of rides in the last hour, you need to aggregate data across multiple rows/columns
    - For such queries, analytical databases are needed, known as Online Analytical Processing (OLAP) databases

- In recent years, trend has shifted away from these
    - Both OLTP and OLAP databases have been combined into databases that support both (CockroachDB, DuckDB etc)
    - Removal of coupling between storage and processing
    - Online processing (i.e. real time processing)

### ETL: Extract Transform, and Load

- ETL is kind of a dated term; in the past you extract the data, transform into some format, and load into a database/data warehouse

- Then came ELT, where you load things as an unprocessed mess into data lake first, then transform it to suit your needs

- These days, solutions are managed hybrids (e.g. Snowflake, Databricks) that provide both data lake and data warehousing services on demand

## How does data flow between processes

### Through Databases

- Only feasible if both processes can access the same DB
- DB Reads/Writes is typically slow, so latency sensitive applications won't be able to do this

### Through REST/RPC APIs

- If you communicate through requests, you need the data to be small and secure enough to be sent via network
- You also need to couple the 2 services tightly. If anything changes in one service in the communication phase, both are impacted
- Nonetheless, microservices are considered best practise today simply because of the ease of setting them up once the contract is finalised

### Through real-time transports (Apache Kafka / Amazon Kinesis)

- API based microservices fail when the request volume gets super large
    - This is because the request model is fundamentally synchronous; the requestor and the receiver both lock until the request is fulfilled
    - When services go down, all data is lost, and all downstream services go down together. Tightly coupled systems!!

- To manage this, newer systems rely on a broker to pass messages to each other
    - Data is broadcasted to the broker when something happens (called an **event**)
    - When an event occurs, data is produced. Systems do not care who is consuming the data, only that it is produced
    - This is a pub/sub system
    - Data will be retained by broker for some pre-specified number of days before being deleted, or moved to permanent storage
    - Example of pubsub solutions are Kafka, Amazon Kinesis, RabbitMQ etc

## Batch vs Stream Processing

- You can either process your data in batch (airflow cron jobs)
    - Lower frequency, but more reliables
    - Suitable for non-time sensitive data
- Or process as a stream (FlinkSQL)
    - Time sensitive data
    - But expensive!!